In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set_theme(palette = "Dark2")
# my_colors = [(0.10588235294117647, 0.6196078431372549, 0.4666666666666667),
#   (0.8509803921568627, 0.37254901960784315, 0.00784313725490196)]
pd.set_option('display.max_columns', None)
from itertools import chain, combinations
from sklearn.preprocessing import LabelEncoder

In [2]:
import importlib
import Model_functions
importlib.reload(Model_functions)
from Model_functions import Model_class 

In [3]:
df = pd.read_csv("df_127_columns.csv")

In [3]:
low_cardinality_cat_cols = ["size","item_color","user_title","user_state"]
high_cardinality_cat_cols = ["item_id","brand_id","user_id"]
cat_cols = low_cardinality_cat_cols + high_cardinality_cat_cols
cat_cols += ["mode_item_id","mode_size","mode_brand_id","mode_item_color",]

In [5]:
reduce_memory = low_cardinality_cat_cols + ["mode_size","mode_item_color"]

In [6]:
# df["size"] = df["size"].astype("category")
# memory = tuple(df.memory_usage(deep=True))
	
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 531170 entries, 0 to 531169
Columns: 127 entries, order_item_id to delivery_time_user_id_mad
dtypes: float64(54), int64(62), object(11)
memory usage: 825.7 MB


In [158]:
df["order_id"] = df["order_id"].astype("category")
df["order_date"] = pd.to_datetime(df["order_date"], format="%Y-%m-%d")
df["delivery_date"] = pd.to_datetime(df["delivery_date"], format="%Y-%m-%d")
df["user_reg_date"] = pd.to_datetime(df["user_reg_date"], format="%Y-%m-%d")
df["user_dob"] = pd.to_datetime(df["user_dob"], format="%Y-%m-%d")
# LE = LabelEncoder()
# for i in reduce_memory:
#     df[i] = LE.fit_transform(df[i])

In [8]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 531170 entries, 0 to 531169
Columns: 127 entries, order_item_id to delivery_time_user_id_mad
dtypes: category(1), datetime64[ns](4), float64(54), int32(6), int64(62)
memory usage: 515.1 MB


In [9]:
# How to shift the date
# df_group[['A', 'B', 'C']] = df_gr.groupby('Name')['A', 'B', 'C'].shift(1)

In [10]:
df_group = df[["order_date","user_id"]].drop_duplicates().sort_values(by=["user_id","order_date"])
df_group["order_date_shifted"] = df_group.groupby("user_id")["order_date"].shift(1)
# df_group[['A', 'B', 'C']] = df_gr.groupby('Name')['A', 'B', 'C'].shift(1)
df_group["days_since_last_order"] = (df_group["order_date"] - df_group["order_date_shifted"]).dt.days
# df_group.fillna(0, inplace=True)
df_group

,order_date,user_id,order_date_shifted,days_since_last_order
24723,2012-04-19,6,NaT,NaN
31011,2012-04-25,6,2012-04-19,6.0
226242,2012-09-23,6,2012-04-25,151.0
240276,2012-10-06,6,2012-09-23,13.0
331324,2013-01-11,6,2012-10-06,97.0
...,...,...,...,...
531141,2013-04-29,91911,NaT,NaN
531149,2013-04-27,91912,NaT,NaN
531154,2013-04-29,91915,NaT,NaN
531158,2013-04-29,91920,NaT,NaN


In [11]:
# get statistics for days since last order
cols_to_agg = ["user_id"]
stats_to_agg = ["mean","median","max","min","std","mad","skew"]
# Loop through the columns to aggregate
for i in cols_to_agg:
    df_group_group = df_group[[i,"days_since_last_order"]].groupby([i]).agg(stats_to_agg).reset_index().round(2)
    df_group_group.columns = [str("_" + i +"_").join(col) for col in df_group_group.columns]
    df_group_group.rename(columns= {i+"_"+i+"_":i},inplace=True)
    df_group = df_group.merge(df_group_group,on=i,how="left")
df_group.fillna(0,inplace=True)
df_group.drop(["order_date_shifted"],axis=1,inplace=True)
df = df.merge(df_group,on=["user_id","order_date"],how="left")    

In [20]:
# Create a powerset of the item descriptions columns
item_descriptions = ["item_id","size","item_color","brand_id"]
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1,len(s)+1))

In [13]:
categorical_powerset = powerset(item_descriptions)
# Use the powerset to create new columns indicating the days since the last order for each combination of the item descriptions
powerset_name_list = []
# exclude powerset elements containing both item_id and brand_id as they are redundant
exclude = ["item_id","brand_id"]
for column in categorical_powerset:
    current = list(column)
    if not all(x in current for x in exclude):
        print(current)
        df_group = df[["order_date","user_id"]+current].drop_duplicates().sort_values(by=["user_id","order_date"])
        df_group["order_date_shifted"] = df_group.groupby(["user_id"] + current)["order_date"].shift(1)
        df_group["days_since_last_order_same"+ "_" + "_".join(current)] = (df_group["order_date"] - df_group["order_date_shifted"]).dt.days
        df_group.fillna(0, inplace=True)
        df_group.drop(["order_date_shifted"],axis=1,inplace=True)
        df = df.merge(df_group,on=["user_id","order_date"]+current,how="left")

['item_id']
['size']
['item_color']
['brand_id']
['item_id', 'size']
['item_id', 'item_color']
['size', 'item_color']
['size', 'brand_id']
['item_color', 'brand_id']
['item_id', 'size', 'item_color']
['size', 'item_color', 'brand_id']


In [14]:
# Get days since last delivery
df_group = df[["delivery_date","user_id"]].drop_duplicates().sort_values(by=["user_id","delivery_date"])
df_group["delivery_date_shifted"] = df_group.groupby("user_id")["delivery_date"].shift(1)
# df_group[['A', 'B', 'C']] = df_gr.groupby('Name')['A', 'B', 'C'].shift(1)
df_group["days_since_last_delivery"] = (df_group["delivery_date"] - df_group["delivery_date_shifted"]).dt.days
# df_group.fillna(0, inplace=True)
df_group

,delivery_date,user_id,delivery_date_shifted,days_since_last_delivery
24723,2012-04-24,6,NaT,NaN
31011,2012-04-26,6,2012-04-24,2.0
31013,2012-05-07,6,2012-04-26,11.0
226242,2012-09-27,6,2012-05-07,143.0
240281,2012-10-10,6,2012-09-27,13.0
...,...,...,...,...
531146,2013-05-20,91911,2013-05-03,17.0
531149,2013-06-10,91912,NaT,NaN
531154,2013-05-03,91915,NaT,NaN
531158,2013-05-03,91920,NaT,NaN


In [15]:
# get statistics for days since last delivery
cols_to_agg = ["user_id"]
stats_to_agg = ["mean","median","max","min","std","mad","skew"]
# Loop through the columns to aggregate
for i in cols_to_agg:
    df_group_group = df_group[[i,"days_since_last_delivery"]].groupby([i]).agg(stats_to_agg).reset_index().round(2)
    df_group_group.columns = [str("_" + i +"_").join(col) for col in df_group_group.columns]
    df_group_group.rename(columns= {i+"_"+i+"_":i},inplace=True)
    df_group = df_group.merge(df_group_group,on=i,how="left")
df_group.fillna(0,inplace=True)
df_group.drop(["delivery_date_shifted"],axis=1,inplace=True)
df = df.merge(df_group,on=["user_id","delivery_date"],how="left")   

In [17]:
categorical_powerset = powerset(item_descriptions)
# Use the powerset to create new columns indicating the days since the last delivery for each combination of the item descriptions
powerset_name_list = []
# exclude powerset elements containing both item_id and brand_id as they are redundant
exclude = ["item_id","brand_id"]
for column in categorical_powerset:
    current = list(column)
    if not all(x in current for x in exclude):
        print(current)
        df_group = df[["delivery_date","user_id"]+current].drop_duplicates().sort_values(by=["user_id","delivery_date"])
        df_group["delivery_date_shifted"] = df_group.groupby(["user_id"] + current)["delivery_date"].shift(1)
        df_group["days_since_last_delivery_same"+ "_" + "_".join(current)] = (df_group["delivery_date"] - df_group["delivery_date_shifted"]).dt.days
        df_group.fillna(0, inplace=True)
        df_group.drop(["delivery_date_shifted"],axis=1,inplace=True)
        df = df.merge(df_group,on=["user_id","delivery_date"]+current,how="left")

['item_id']
['size']
['item_color']
['brand_id']
['item_id', 'size']
['item_id', 'item_color']
['size', 'item_color']
['size', 'brand_id']
['item_color', 'brand_id']
['item_id', 'size', 'item_color']
['size', 'item_color', 'brand_id']


In [19]:
# !! Include in code base
# Use the numeric variable order_sum to create a new categorical variable using bins
bins_range = df["order_sum"].describe(percentiles=[0.1,0.25,0.5,0.75,0.9])
# bins_range
bins = bins_range[["min","10%","25%","50%","75%","90%","max"]].values
df['order_sum_bins'] = pd.cut(df['order_sum'], bins=bins, include_lowest=True).cat.codes
bins_range = df["item_price"].describe(percentiles=[0.1,0.25,0.5,0.75,0.95])
bins = bins_range[["min","10%","25%","50%","75%","95%","max"]].values
df['item_price_bins'] = pd.cut(df['item_price'], bins=bins, include_lowest=True).cat.codes

In [20]:
bins = [20,24,29,34,44,55,65,85]
df['age_bins'] = pd.cut(df['user_age'], bins=bins, include_lowest=True).cat.codes

In [21]:
df["user_reg_age"].describe(percentiles=[0.1,0.25,0.5,0.75,0.9])
bins = [-1,0,0.1,7,14,30,60,90,180,365,730,805]
df['reg_age_bins'] = pd.cut(df['user_reg_age'], bins=bins, include_lowest=True).cat.codes

In [22]:
bins_column_list = ["order_sum_bins","item_price_bins","age_bins","reg_age_bins"]

In [23]:
cols_to_agg = bins_column_list + ["order_weekday", "delivery_weekday"]
stats_to_agg = ["mean","std","min","max","median","mad","skew"]
# Loop through the columns to aggregate
for i in cols_to_agg:
    df_group = df[[i,"delivery_time"]].groupby([i]).agg(stats_to_agg).reset_index().round(2)
    df_group.columns = [str("_" + i +"_").join(col) for col in df_group.columns]
    df_group.rename(columns= {i+"_"+i+"_":i},inplace=True)
    df_group.fillna(0,inplace=True)
    df = df.merge(df_group,on=i,how="left")   

In [24]:
df.drop(bins_column_list,axis=1,inplace=True)

In [8]:
def powerset(iterable,r1,r2):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(r1,r2))

In [ ]:
# Interactions between delivery time and categorical variables
interactions_list = ["item_id","brand_id","item_color","size","delivery_weekday","user_id"]
categorical_powerset = powerset(interactions_list,2,3)
# Use the powerset to create new columns indicating the days since the last delivery for each combination of the item descriptions
stats_to_agg = ["mean","std","min","max","median","mad","skew"]
# exclude powerset elements containing both item_id and brand_id as they are redundant
exclude = ["item_id","brand_id"]
for column in categorical_powerset:
    i = list(column)
    if not all(x in i for x in exclude):
        print(i)
        df_group = df[i+["delivery_time"]].groupby(i).agg(stats_to_agg).reset_index().round(2)
        df_group.columns = [str("_" + i[0] +"_" + i[1] + "_").join(col) for col in df_group.columns]
        df_group.rename(columns= {df_group.columns[0]:str(i[0]), df_group.columns[1]:str(i[1])},inplace=True)
        df_group.fillna(0,inplace=True)
        df = df.merge(df_group,on=i,how="left") 

In [78]:
# Interactions between delivery time and categorical variables
interactions_list = ["item_id","brand_id","item_color","size","delivery_weekday","user_id"]
categorical_powerset = powerset(interactions_list,3,4)
# Use the powerset to create new columns indicating the days since the last delivery for each combination of the item descriptions
stats_to_agg = ["mean","std","min","max","median","mad","skew"]
# exclude powerset elements containing both item_id and brand_id as they are redundant
exclude = ["item_id","brand_id"]
for column in categorical_powerset:
    i = list(column)
    if not all(x in i for x in exclude):
        print(i)
        df_group = df[i+["delivery_time"]].groupby(i).agg(stats_to_agg).reset_index().round(2)
        df_group.columns = [str("_" + i[0] +"_" + i[1] + "_" + i[2] + "_").join(col) for col in df_group.columns]
        df_group.rename(columns= {df_group.columns[0]:str(i[0]), df_group.columns[1]:str(i[1]),df_group.columns[2]:str(i[2])},inplace=True)
        df_group.fillna(0,inplace=True)
        df = df.merge(df_group,on=i,how="left") 

['item_id', 'item_color', 'size']
['item_id', 'item_color', 'delivery_weekday']
['item_id', 'item_color', 'user_id']
['item_id', 'size', 'delivery_weekday']
['item_id', 'size', 'user_id']
['item_id', 'delivery_weekday', 'user_id']
['brand_id', 'item_color', 'size']
['brand_id', 'item_color', 'delivery_weekday']
['brand_id', 'item_color', 'user_id']
['brand_id', 'size', 'delivery_weekday']
['brand_id', 'size', 'user_id']
['brand_id', 'delivery_weekday', 'user_id']
['item_color', 'size', 'delivery_weekday']
['item_color', 'size', 'user_id']
['item_color', 'delivery_weekday', 'user_id']
['size', 'delivery_weekday', 'user_id']


In [118]:
# Get order sum stats
# cols_to_agg = ["item_id","size","brand_id","item_color","user_id"]
# stats_to_agg = ["mean","std","min","max","median","mad"]
# # Loop through the columns to aggregate
# for i in cols_to_agg:
#     df_group = df[[i,"order_sum"]].groupby([i]).agg(stats_to_agg).reset_index().round(2)
#     df_group.columns = [str("_" + i +"_").join(col) for col in df_group.columns]
#     df_group.rename(columns= {i+"_"+i+"_":i},inplace=True)
#     df_group.fillna(0,inplace=True)
#     df = df.merge(df_group,on=i,how="left")    

In [70]:
# df.to_csv("df_last.csv",index=False)

In [ ]:
# look_for = ["max","min","mean","median","mad"]
# get_columns_delivery_time = [col for col in df.columns if "delivery_time" in col]
# get_columns_delivery_time_look_for = [col for col in get_columns_delivery_time if any(x in col for x in look_for)]
# get_columns_delivery_time_look_for

In [ ]:
# # Get the delivery features which are in the top 115 of the feature importance
# !!! Code base
# tuple_list = [item for t in feature_importance[:100] for item in t]
# most_important = [ col for col in get_columns_delivery_time_look_for if col in tuple_list[:115]]
# most_important

In [109]:
# for i in most_important:
#     df[i+"_diff"] =  df["delivery_time"]/df[i]

In [86]:
# df.to_csv("df_last_X3.csv",index=False)

In [4]:
df = pd.read_csv("df_last_X3.csv")

In [5]:
# Total items ordered by user
df_group = df[["user_id","user_state"]].groupby(["user_id"]).count().reset_index()
df_group.rename(columns= {"user_state":"user_id_total_items_ordered"},inplace=True)
# Use join instead of merge to speed up the process as merge is slower CODEBASE INCLUDE !!!
df = df.join(df_group.set_index("user_id"),on="user_id",how="left")

In [6]:
# Cumulative sum of items ordered by user per order day
df_group = df[["user_id","user_state","order_date"]].groupby(["user_id","order_date"]).count().reset_index()
df_group.rename(columns= {"user_state":"user_id_total_items_ordered"},inplace=True)
df_group["user_id_total_items_ordered_cumsum"] = df_group.groupby(["user_id"])["user_id_total_items_ordered"].cumsum()
df_group.drop(columns=["user_id_total_items_ordered"],inplace=True)
df = df.join(df_group.set_index(["user_id","order_date"]),on=["user_id","order_date"],how="left")

In [103]:
# correct
df_group = df[["user_id","user_state","order_date","item_id"]].groupby(["user_id","item_id","order_date"]).count().reset_index()
df_group["user_id_total_items_ordered_cumsum"] = df_group.groupby(["user_id","item_id"])["user_state"].cumsum()
df_group

,user_id,item_id,order_date,user_state,user_id_total_items_ordered_cumsum
0,6,96,2012-04-25,3,3
1,6,250,2012-09-23,1,1
2,6,261,2012-04-19,1,1
3,6,320,2012-09-23,1,1
4,6,1404,2012-09-23,3,3
...,...,...,...,...,...
426359,91920,2489,2013-04-29,1,1
426360,91920,2505,2013-04-29,2,2
426361,91920,2547,2013-04-29,1,1
426362,91920,2591,2013-04-29,1,1


In [9]:
# Total items ordered by user and interactions
interactions_list = ["item_id","brand_id","item_color","size"]
categorical_powerset = powerset(interactions_list,1,len(interactions_list)+1)
exclude = ["item_id","brand_id"]
for column in categorical_powerset:
    i = list(column)
    if not all(x in i for x in exclude):
        print(i)
        df_group = df[["user_id","user_state"]+i].groupby(["user_id"]+i).count().reset_index()
        df_group.rename(columns= {"user_state":"user_id_"+ "_".join(i) + "_total_items_ordered"},inplace=True)
        # Use join instead of merge to speed up the process as merge is slower CODEBASE INCLUDE !!!
        df = df.join(df_group.set_index(["user_id"]+i),on=["user_id"]+i,how="left")
        

['item_id']
['brand_id']
['item_color']
['size']
['item_id', 'item_color']
['item_id', 'size']
['brand_id', 'item_color']
['brand_id', 'size']
['item_color', 'size']
['item_id', 'item_color', 'size']
['brand_id', 'item_color', 'size']


In [10]:
# Cumulative sum of items ordered by user per order day
interactions_list = ["item_id","brand_id","item_color","size"]
categorical_powerset = powerset(interactions_list,1,len(interactions_list)+1)
exclude = ["item_id","brand_id"]
for column in categorical_powerset:
    i = list(column)
    if not all(x in i for x in exclude):
        print(i)
        df_group = df[["user_id","user_state","order_date"]+i].groupby(["user_id"]+i+["order_date"]).count().reset_index()
        df_group["user_state"] = df_group.groupby(["user_id"]+i)["user_state"].cumsum()
        df_group.rename(columns= {"user_state":"user_id_"+ "_".join(i) + "_cumsum_items_ordered"},inplace=True)
        # Use join instead of merge to speed up the process as merge is slower CODEBASE INCLUDE !!!
        df = df.join(df_group.set_index(["user_id","order_date"]+i),on=["user_id","order_date"]+i,how="left")

['item_id']
['brand_id']
['item_color']
['size']
['item_id', 'item_color']
['item_id', 'size']
['brand_id', 'item_color']
['brand_id', 'size']
['item_color', 'size']
['item_id', 'item_color', 'size']
['brand_id', 'item_color', 'size']


In [192]:
# df_group = df[["user_id","user_state","order_date"]+["item_id"]].groupby(["user_id"]+["item_id"]+["order_date"]).count()
# # df_group["last_order_count"] = df_group.groupby("user_id","item_id",,"order_date")["user_state"].shift(1)
# # df_group[df_group["user_id"] == 6]
# df_group

user_state
user_id item_id order_date            
6       96      2012-04-25           3
        250     2012-09-23           1
        261     2012-04-19           1
        320     2012-09-23           1
        1404    2012-09-23           3
...                                ...
91920   2489    2013-04-29           1
        2505    2013-04-29           2
        2547    2013-04-29           1
        2591    2013-04-29           1
91922   2452    2013-04-28           2

[426364 rows x 1 columns]

In [ ]:
df_group = df[["user_id","user_state","order_date"]+["item_id"]].groupby(["user_id"]+["item_id"]+["order_date"]).count().reset_index()
df_group
# df_group["last_order_count_same_item_id"] = df_group.groupby(["user_id","item_id"])["user_state"].shift(1)
# df_group[df_group["user_id"] == 2649]

In [ ]:
# # Number of same items in previous order no increase
# interactions_list = ["item_id","brand_id","item_color","size"]
# categorical_powerset = powerset(interactions_list,1,len(interactions_list)+1)
# exclude = ["item_id","brand_id"]
# for column in categorical_powerset:
#     i = list(column)
#     if not all(x in i for x in exclude):
#         print(i)
#         df_group = df[["user_id","user_state","order_date"]+i].groupby(["user_id"]+i+["order_date"]).count().reset_index()
#         # df_group["user_state"] = df_group.groupby(["user_id"]+i)["user_state"].count()
#         # df_group.rename(columns= {"user_state":"user_id_last_order_count_same" + "_".join(i)},inplace=True)
#         column_name = "user_id_last_order_count_same_" + "_".join(i)
#         df_group[column_name] = df_group.groupby(["user_id"]+i)["user_state"].shift(1)
#         df_group = df_group.dropna()
#         # Use join instead of merge to speed up the process as merge is slower CODEBASE INCLUDE !!!
#         df = df.join(df_group[["user_id","order_date"]+i+[column_name]].set_index(["user_id","order_date"]+i),on=["user_id","order_date"]+i,how="left")

In [ ]:
# to_fill = df.loc[:,"user_id_last_order_count_sameitem_id":].columns
# df[to_fill] = df[to_fill].fillna(0)
# df

In [133]:
# # 2nd degree interaction user id brand item id item price statts
# # Interactions between delivery time and categorical variables
# interactions_list = ["item_id","brand_id","item_color","size","user_id"]
# categorical_powerset = powerset(interactions_list,2,3)
# # Use the powerset to create new columns indicating the days since the last delivery for each combination of the item descriptions
# stats_to_agg = ["mean","std","min","max","median","mad","skew"]
# # exclude powerset elements containing both item_id and brand_id as they are redundant
# exclude = ["item_id","brand_id"]
# for column in categorical_powerset:
#     i = list(column)
#     if not all(x in i for x in exclude):
#         print(i)
#         df_group = df[i+["item_price"]].groupby(i).agg(stats_to_agg).reset_index().round(2)
#         df_group.columns = [str("_" + i[0] +"_" + i[1] + "_").join(col) for col in df_group.columns]
#         df_group.rename(columns= {df_group.columns[0]:str(i[0]), df_group.columns[1]:str(i[1])},inplace=True)
#         df_group.fillna(0,inplace=True)
#         df = df.merge(df_group,on=i,how="left")

['item_id', 'item_color']
['item_id', 'size']
['item_id', 'user_id']
['brand_id', 'item_color']
['brand_id', 'size']
['brand_id', 'user_id']
['item_color', 'size']
['item_color', 'user_id']
['size', 'user_id']


In [ ]:
# create cartesian product of all combinations of the columns in the list
# cartesian_product = test_left.merge(test_right, how="cross")
# cartesian_product.drop(columns=["return_x"],inplace=True)
# cartesian_product.rename(columns={"return_y":"return"},inplace=True)
# cartesian_product.drop_duplicates(inplace=True)
# cartesian_product

In [ ]:
# cartesian_product["days_difference"] = cartesian_product["order_date"] - cartesian_product["delivery_date"]
# cartesian_product["days_difference"] = cartesian_product["days_difference"].dt.days
# cartesian_product

In [376]:
# Select only the rows where the difference between the order date and delivery date is more than 14 days
# cartesian_product = cartesian_product[cartesian_product["days_difference"] > 30]
# cartesian_product_group = cartesian_product[["order_date","return"]].groupby(["order_date"]).sum()
# cartesian_product_group
# # cartesian_product

,return
order_date,
2012-09-23,1
2012-10-06,1
2013-01-11,7


In [381]:
# cartesian_product_group.reset_index()
# cartesian_product_group["user_id"] = 6
# cartesian_product_group

,return,user_id
order_date,,
2012-09-23,1,6
2012-10-06,1,6
2013-01-11,7,6


In [485]:
def compicated_function(df):
    df_group = df[["user_id","return","order_date","delivery_date"]].where(df["return"]==1).groupby(["user_id","delivery_date","order_date"]).count()
    df_group = df_group.reset_index(level=["delivery_date","order_date"])
    list_of_indexes = df_group.index.unique()
    cartesian_product_group_placeholder = pd.DataFrame(columns=["user_id","order_date","returned_last_deliveries_sum"])
    for i in range(0,len(list_of_indexes)):
        if len(df_group.loc[list_of_indexes[i]]) > 3:
            cartesian_product = df_group.loc[list_of_indexes[i]]
            cartesian_product = cartesian_product[["order_date","return"]].merge(cartesian_product[["delivery_date","return"]], how="cross")
            
            cartesian_product.drop(columns=["return_x"],inplace=True)
            cartesian_product.rename(columns={"return_y":"returned_last_deliveries_sum"},inplace=True)
            cartesian_product.drop_duplicates(inplace=True)
            cartesian_product["days_difference"] = cartesian_product["order_date"] - cartesian_product["delivery_date"]
            cartesian_product["days_difference"] = cartesian_product["days_difference"].dt.days
            
            cartesian_product = cartesian_product[cartesian_product["days_difference"] > 30]
            cartesian_product_group = cartesian_product[["order_date","returned_last_deliveries_sum"]].groupby(["order_date"]).sum()
            cartesian_product_group.reset_index(inplace=True)
            cartesian_product_group["user_id"] = list_of_indexes[i]
            
            cartesian_product_group_placeholder = pd.concat([cartesian_product_group_placeholder,cartesian_product_group])
    
    cartesian_product_group_placeholder.set_index(["user_id","order_date"],inplace=True)
    # df = df.join(cartesian_product_group_placeholder,on=["user_id","order_date"],how="left")
    return cartesian_product_group_placeholder

In [486]:
# cartesian_product_group_placeholder = compicated_function(df)

In [490]:
cartesian_product_group_placeholder

returned_last_deliveries_sum
user_id order_date                             
6.0     2012-09-23                            1
        2012-10-06                            1
        2013-01-11                            7
9.0     2012-12-31                            4
        2013-01-09                            4
...                                         ...
74045.0 2013-03-26                            2
74133.0 2013-03-26                            2
75019.0 2013-03-24                            5
75076.0 2013-03-24                            1
75567.0 2013-03-25                            7

[25751 rows x 1 columns]

In [491]:
df = df.join(cartesian_product_group_placeholder,on=["user_id","order_date"],how="left")
# df = pd.merge(df,cartesian_product_group_placeholder,how="left",on=["user_id","order_date"])

In [ ]:
# df_group = df[["user_id","return","order_date","delivery_date"]].where(df["return"]==1).groupby(["user_id","delivery_date","order_date"]).count()
# df_group = df_group.reset_index(level=["delivery_date","order_date"])
# cartesian_product_group_placeholder = pd.DataFrame(columns=["user_id","order_date","returned_last_deliveries_sum"])
# cartesian_product = df_group.loc[df_group.index.unique()[1]]
# cartesian_product = cartesian_product[["order_date","return"]].merge(cartesian_product[["delivery_date","return"]], how="cross")

# cartesian_product.drop(columns=["return_x"],inplace=True)
# cartesian_product.rename(columns={"return_y":"returned_last_deliveries_sum"},inplace=True)
# cartesian_product.drop_duplicates(inplace=True)

# cartesian_product["days_difference"] = cartesian_product["order_date"] - cartesian_product["delivery_date"]
# cartesian_product["days_difference"] = cartesian_product["days_difference"].dt.days

# cartesian_product = cartesian_product[cartesian_product["days_difference"] > 30]
# cartesian_product_group = cartesian_product[["order_date","returned_last_deliveries_sum"]].groupby(["order_date"]).sum()
# cartesian_product_group.reset_index(inplace=True)
# cartesian_product_group["user_id"] = 9
# cartesian_product_group_placeholder = pd.concat([cartesian_product_group_placeholder,cartesian_product_group])
# cartesian_product_group_placeholder.set_index(["user_id","order_date"],inplace=True)
# cartesian_product_group_placeholder

# df = df.join(cartesian_product_group,on=["user_id","order_date"],how="left")
# cartesian_product_group

In [ ]:
# n unique for user id
# interaction_list = ["item_id","size","item_color","brand_id"]
# current = ["brand_id","size"]
# df_group = df[["user_id","item_id","size","item_color","brand_id"]].groupby(["user_id"]).nunique()
# # df_group.columns = ["user_id_"+i+"_nunique" for i in interaction_list]
# # df_group.columns = ["user_id_brand_id_size_item_color_nunique"]
# # df.group.columns = ["user_id_brand_id_size_item_color_nunique"]
# df_group.columns = ["user_id_"+i+"_nunique_" +"_".join(current)  for i in df_group.columns]
# df_group.set_index("user_id")
# df_group

In [82]:
# n uniqie for user id
interactions_list = ["item_id","brand_id","item_color","size"]
categorical_powerset = powerset(interactions_list,0,len(interactions_list)-1)
exclude = ["item_id","brand_id"]
for column in categorical_powerset:
    current = list(column)
    if not all(x in current for x in exclude) and current != ['item_id', 'item_color', 'size']:
        print(current)
        df_group = df[["user_id","item_id","size","item_color","brand_id"]].groupby(["user_id"]+current).nunique()
        df_group.columns = ["user_id_"+i+"_nunique_" +"_".join(current)  for i in df_group.columns]
        # Use join instead of merge to speed up the process as merge is slower CODEBASE INCLUDE !!!
        df = df.join(df_group,on=["user_id"]+current,how="left")

[]
['item_id']
['brand_id']
['item_color']
['size']
['item_id', 'item_color']
['item_id', 'size']
['brand_id', 'item_color']
['brand_id', 'size']
['item_color', 'size']
['brand_id', 'item_color', 'size']


In [232]:
df_group = df[["user_id","item_id","size","item_color","brand_id"]].groupby(["size"]).nunique()
df_group

,user_id,item_id,item_color,brand_id
size,,,,
0,20431,1408,77,84
1,29319,1865,78,99
2,33657,1952,78,101
3,25948,1333,77,129
4,12491,886,63,63
5,18983,1818,81,95
6,2686,205,32,28
7,1913,95,25,14


In [230]:
df

,order_item_id,order_date,delivery_date,item_id,size,item_color,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date,return,delivery_time,order_id,user_age,user_reg_age,order_weekday,delivery_weekday,order_month,delivery_month,order_day,delivery_day,order_week,delivery_week,order_item_count,order_sum,average_item_price_order,order_number_same_item_id,order_number_different_item_id,order_number_same_size,order_number_different_size,order_number_same_item_color,order_number_different_item_color,order_number_same_brand_id,order_number_different_brand_id,order_number_same_item_id_size,order_number_different_item_id_size,order_number_same_item_id_item_color,order_number_different_item_id_item_color,order_number_same_size_item_color,order_number_different_size_item_color,order_number_same_size_brand_id,order_number_different_size_brand_id,order_number_same_item_color_brand_id,order_number_different_item_color_brand_id,order_number_same_item_id_size_item_color,order_number_different_item_id_size_item_color,order_number_same_size_item_color_brand_id,order_number_different_size_item_color_brand_id,order_item_id_nunique,order_size_nunique,order_brand_id_nunique,order_item_color_nunique,order_item_id_color_nunique,order_item_id_size_nunique,order_brand_id_color_nunique,order_brand_id_size_nunique,order_brand_id_item_id_nunique,item_price_item_id_mean,item_price_item_id_std,item_price_item_id_min,item_price_item_id_max,item_price_item_id_sum,item_price_item_id_count,item_price_item_id_median,item_price_item_id_mad,item_price_user_id_mean,item_price_user_id_std,item_price_user_id_min,item_price_user_id_max,item_price_user_id_sum,item_price_user_id_count,item_price_user_id_median,item_price_user_id_mad,item_price_brand_id_mean,item_price_brand_id_std,item_price_brand_id_min,item_price_brand_id_max,item_price_brand_id_sum,item_price_brand_id_count,item_price_brand_id_median,item_price_brand_id_mad,price-item_price_item_id_mean,price-item_price_item_id_min,price-item_price_item_id_max,price-item_price_user_id_mean,price-item_price_user_id_min,price-item_price_user_id_max,price-item_price_brand_id_mean,price-item_price_brand_id_min,price-item_price_brand_id_max,mode_item_id,mode_size,mode_brand_id,mode_item_color,delivery_time_item_id_mean,delivery_time_item_id_std,delivery_time_item_id_min,delivery_time_item_id_max,delivery_time_item_id_median,delivery_time_item_id_mad,delivery_time_size_mean,delivery_time_size_std,delivery_time_size_min,delivery_time_size_max,delivery_time_size_median,delivery_time_size_mad,delivery_time_brand_id_mean,delivery_time_brand_id_std,delivery_time_brand_id_min,delivery_time_brand_id_max,delivery_time_brand_id_median,delivery_time_brand_id_mad,delivery_time_item_color_mean,delivery_time_item_color_std,delivery_time_item_color_min,delivery_time_item_color_max,delivery_time_item_color_median,delivery_time_item_color_mad,delivery_time_user_id_mean,delivery_time_user_id_std,delivery_time_user_id_min,delivery_time_user_id_max,delivery_time_user_id_median,delivery_time_user_id_mad,days_since_last_order,days_since_last_order_user_id_mean,days_since_last_order_user_id_median,days_since_last_order_user_id_max,days_since_last_order_user_id_min,days_since_last_order_user_id_std,days_since_last_order_user_id_mad,days_since_last_order_user_id_skew,days_since_last_order_same_item_id,days_since_last_order_same_size,days_since_last_order_same_item_color,days_since_last_order_same_brand_id,days_since_last_order_same_item_id_size,days_since_last_order_same_item_id_item_color,days_since_last_order_same_size_item_color,days_since_last_order_same_size_brand_id,days_since_last_order_same_item_color_brand_id,days_since_last_order_same_item_id_size_item_color,days_since_last_order_same_size_item_color_brand_id,days_since_last_delivery,days_since_last_delivery_user_id_mean,days_since_last_delivery_user_id_median,days_since_last_delivery_user_id_max,days_since_last_delivery_user_id_min,days_since_last_delivery_user_id_st

In [80]:
#  to_drop = df.loc[:,"user_id_item_id_nunique_":].columns
#  df.drop(to_drop,axis=1,inplace=True)

In [227]:
# number of same item_id previous orders
# df_group = df[["user_id","item_id","order_date"]].groupby(["user_id","item_id"]).count()
# df_group = df[["order_date","user_id","item_id"]].sort_values(by=["user_id","order_date"]).set_index(["user_id","order_date"])
# df_group 
# df_group

In [ ]:
# df_group = df[["user_id","item_id","order_date"]].groupby(["user_id","item_id"]).count()
# df_group = df[["order_date","user_id","item_id"]].sort_values(by=["user_id","order_date"]).set_index(["user_id"])
# df_group
# cartesian_product_group_placeholder = pd.DataFrame(columns=["item_id","order_date","ordered_same_item_id_previous_orders"])
# cartesian_product = df_group.loc[df_group.index.unique()[0]]
# cartesian_product["item_id_count"] = cartesian_product["item_id"].map(cartesian_product["item_id"].value_counts())
# cartesian_product.drop_duplicates(inplace=True)
# cartesian_product
# order_date = cartesian_product[["order_date"]].reset_index(drop=True).drop_duplicates()
# order_date["last_order_date"] = order_date["order_date"].shift(1)
# cartesian_product = cartesian_product.merge(order_date,on="order_date",how="left")
# cartesian_product["days_difference"] = cartesian_product["order_date"] - cartesian_product["last_order_date"]


# cartesian_product

# cartesian_product.drop(columns=["return_x"],inplace=True)
# cartesian_product.rename(columns={"return_y":"returned_last_deliveries_sum"},inplace=True)
# cartesian_product.drop_duplicates(inplace=True)

# cartesian_product["days_difference"] = cartesian_product["order_date"] - cartesian_product["delivery_date"]
# cartesian_product["days_difference"] = cartesian_product["days_difference"].dt.days

# cartesian_product = cartesian_product[cartesian_product["days_difference"] > 30]
# cartesian_product_group = cartesian_product[["order_date","returned_last_deliveries_sum"]].groupby(["order_date"]).sum()
# cartesian_product_group.reset_index(inplace=True)
# cartesian_product_group["user_id"] = 9
# cartesian_product_group_placeholder = pd.concat([cartesian_product_group_placeholder,cartesian_product_group])
# cartesian_product_group_placeholder.set_index(["user_id","order_date"],inplace=True)
# cartesian_product_group_placeholder

In [221]:
# Instantiate the model class
Model = Model_class(df)

In [222]:
# Split the data into train and test sets
df_train,df_test, df_valid,k = Model.split_data(False)

In [136]:
# Encode the categorical variables using leave-one-out encoding
df_train_encoded,df_test_encoded,encoder = Model.LOE_Encoder(df_train,df_test, cat_cols,0.1,True)

In [139]:
# # Encode the categorical variables using leave-one-out encoding
# df_train_encoded,df_test_encoded,encoder = Model.LOE_Encoder(df_train_whole,df_valid, cat_cols,0.1,True)

In [119]:
catboost_params_cpu = {'iterations': 150, 'learning_rate': 0.1, 'depth': 10, 'loss_function': 'Logloss', 'random_seed': 42}
catboost_model_cpu, catboost_Y_pred_cpu, catboost_mae_cpu, catboost_Y_pred_proba_cpu = Model.catboost(df_train,df_test,catboost_params_cpu,1,cat_cols)
print("Catboost_cpu MAE on test set:", catboost_mae_cpu)

0:	learn: 0.6765302	test: 0.6791931	best: 0.6791931 (0)	total: 2.63s	remaining: 6m 32s
1:	learn: 0.6628056	test: 0.6672090	best: 0.6672090 (1)	total: 5.04s	remaining: 6m 12s
2:	learn: 0.6467568	test: 0.6552143	best: 0.6552143 (2)	total: 7.64s	remaining: 6m 14s
3:	learn: 0.6343216	test: 0.6458452	best: 0.6458452 (3)	total: 9.92s	remaining: 6m 2s
4:	learn: 0.6244133	test: 0.6384065	best: 0.6384065 (4)	total: 12.1s	remaining: 5m 51s
5:	learn: 0.6162083	test: 0.6321311	best: 0.6321311 (5)	total: 14.5s	remaining: 5m 47s
6:	learn: 0.6091435	test: 0.6269395	best: 0.6269395 (6)	total: 16.8s	remaining: 5m 43s
7:	learn: 0.6033483	test: 0.6227330	best: 0.6227330 (7)	total: 19.3s	remaining: 5m 42s
8:	learn: 0.5986944	test: 0.6191776	best: 0.6191776 (8)	total: 21.6s	remaining: 5m 37s
9:	learn: 0.5942079	test: 0.6157490	best: 0.6157490 (9)	total: 24.3s	remaining: 5m 39s
10:	learn: 0.5905508	test: 0.6130183	best: 0.6130183 (10)	total: 27s	remaining: 5m 40s
11:	learn: 0.5877868	test: 0.6107960	best: 0

In [223]:
# concat df_train and df_test
df_train_whole = pd.concat([df_train,df_test],axis=0)

In [120]:
catboost_params_cpu = {'iterations': 150, 'learning_rate': 0.1, 'depth': 10, 'loss_function': 'Logloss', 'random_seed': 42}
catboost_model_cpu_valid, catboost_Y_pred_cpu_valid, catboost_mae_cpu_valid, catboost_Y_pred_proba_cpu_valid = Model.catboost(df_train_whole,df_valid,catboost_params_cpu,1,cat_cols)
print("Catboost_cpu MAE on test set:", catboost_mae_cpu_valid)

0:	learn: 0.6762372	test: 0.6766548	best: 0.6766548 (0)	total: 2.76s	remaining: 6m 51s
1:	learn: 0.6623819	test: 0.6631706	best: 0.6631706 (1)	total: 5.29s	remaining: 6m 31s
2:	learn: 0.6513204	test: 0.6525817	best: 0.6525817 (2)	total: 8.19s	remaining: 6m 41s
3:	learn: 0.6389138	test: 0.6418496	best: 0.6418496 (3)	total: 10.8s	remaining: 6m 33s
4:	learn: 0.6284856	test: 0.6336643	best: 0.6336643 (4)	total: 13.4s	remaining: 6m 29s
5:	learn: 0.6200529	test: 0.6273519	best: 0.6273519 (5)	total: 15.8s	remaining: 6m 20s
6:	learn: 0.6121431	test: 0.6210798	best: 0.6210798 (6)	total: 18.3s	remaining: 6m 13s
7:	learn: 0.6049819	test: 0.6153917	best: 0.6153917 (7)	total: 20.8s	remaining: 6m 8s
8:	learn: 0.5997404	test: 0.6110703	best: 0.6110703 (8)	total: 23.4s	remaining: 6m 6s
9:	learn: 0.5951701	test: 0.6075777	best: 0.6075777 (9)	total: 26.1s	remaining: 6m 4s
10:	learn: 0.5910698	test: 0.6037568	best: 0.6037568 (10)	total: 28.4s	remaining: 5m 59s
11:	learn: 0.5880491	test: 0.6015626	best: 0

In [ ]:
Model.get_feature_importance(catboost_model_cpu,df_valid)

In [225]:
Model.get_feature_importance(catboost_model_gpu,df_valid)

[('user_id', 9.45201277436173),
 ('delivery_time', 8.377757039287646),
 ('item_id', 6.095525402414859),
 ('user_id_item_id_item_color_total_items_ordered', 2.973352165001188),
 ('user_id_size_nunique_item_id_item_color', 2.3856539267611683),
 ('days_since_last_delivery', 1.8546444422625499),
 ('order_sum', 1.5890228763781642),
 ('delivery_time_item_id_delivery_weekday_user_id_max', 1.4299448674324096),
 ('item_price', 1.336473602183114),
 ('days_since_last_delivery_same_item_id', 1.3362757520037196),
 ('delivery_time_delivery_weekday_std', 1.2708233793108028),
 ('delivery_time_item_id_delivery_weekday_user_id_min', 1.2218311475897408),
 ('delivery_time_delivery_weekday_mad', 1.1979615178588723),
 ('days_since_last_order_same_item_id', 1.181046045507506),
 ('price-item_price_item_id_mean', 0.9571649585585623),
 ('delivery_time_order_sum_bins_std', 0.8884136896422828),
 ('brand_id', 0.8741699333778234),
 ('user_id_size_nunique_item_id', 0.8069905281790698),
 ('delivery_time_item_id_item_

In [73]:
catboost_params_gpu = {'iterations': 175, 'learning_rate': 0.11, 'depth': 10, 'loss_function': 'Logloss', 'random_seed': 42,"task_type":"GPU"}
catboost_model_gpu, catboost_Y_pred_gpu, catboost_mae_gpu, catboost_Y_pred_proba_gpu = Model.catboost(df_train,df_test,catboost_params_gpu,1,cat_cols)
print("Catboost_gpu MAE on test set:", catboost_mae_gpu)

0:	learn: 0.6747637	test: 0.6774733	best: 0.6774733 (0)	total: 520ms	remaining: 1m 30s
1:	learn: 0.6598699	test: 0.6644793	best: 0.6644793 (1)	total: 990ms	remaining: 1m 25s
2:	learn: 0.6453004	test: 0.6544781	best: 0.6544781 (2)	total: 1.42s	remaining: 1m 21s
3:	learn: 0.6328014	test: 0.6453921	best: 0.6453921 (3)	total: 1.88s	remaining: 1m 20s
4:	learn: 0.6236320	test: 0.6388230	best: 0.6388230 (4)	total: 2.31s	remaining: 1m 18s
5:	learn: 0.6145154	test: 0.6320417	best: 0.6320417 (5)	total: 2.87s	remaining: 1m 20s
6:	learn: 0.6073080	test: 0.6270478	best: 0.6270478 (6)	total: 3.35s	remaining: 1m 20s
7:	learn: 0.6013197	test: 0.6230860	best: 0.6230860 (7)	total: 3.82s	remaining: 1m 19s
8:	learn: 0.5970213	test: 0.6206269	best: 0.6206269 (8)	total: 4.27s	remaining: 1m 18s
9:	learn: 0.5922174	test: 0.6172900	best: 0.6172900 (9)	total: 4.74s	remaining: 1m 18s
10:	learn: 0.5891928	test: 0.6153939	best: 0.6153939 (10)	total: 5.21s	remaining: 1m 17s
11:	learn: 0.5848544	test: 0.6116763	best

In [113]:
catboost_params_gpu = {'iterations': 175, 'learning_rate': 0.11, 'depth': 10, 'loss_function': 'Logloss', 'random_seed': 42,"task_type":"GPU"}
catboost_model_gpu, catboost_Y_pred_gpu, catboost_mae_gpu, catboost_Y_pred_proba_gpu = Model.catboost(df_train,df_test,catboost_params_gpu,1,cat_cols)
print("Catboost_gpu MAE on test set:", catboost_mae_gpu)

0:	learn: 0.6747053	test: 0.6773704	best: 0.6773704 (0)	total: 561ms	remaining: 1m 37s
1:	learn: 0.6596193	test: 0.6640274	best: 0.6640274 (1)	total: 1.02s	remaining: 1m 28s
2:	learn: 0.6439748	test: 0.6523762	best: 0.6523762 (2)	total: 1.51s	remaining: 1m 26s
3:	learn: 0.6316510	test: 0.6433500	best: 0.6433500 (3)	total: 1.97s	remaining: 1m 24s
4:	learn: 0.6226643	test: 0.6373507	best: 0.6373507 (4)	total: 2.42s	remaining: 1m 22s
5:	learn: 0.6142357	test: 0.6313805	best: 0.6313805 (5)	total: 2.89s	remaining: 1m 21s
6:	learn: 0.6080120	test: 0.6273345	best: 0.6273345 (6)	total: 3.33s	remaining: 1m 19s
7:	learn: 0.6015996	test: 0.6229345	best: 0.6229345 (7)	total: 3.84s	remaining: 1m 20s
8:	learn: 0.5976398	test: 0.6202883	best: 0.6202883 (8)	total: 4.36s	remaining: 1m 20s
9:	learn: 0.5937463	test: 0.6176842	best: 0.6176842 (9)	total: 4.86s	remaining: 1m 20s
10:	learn: 0.5892146	test: 0.6136999	best: 0.6136999 (10)	total: 5.29s	remaining: 1m 18s
11:	learn: 0.5856499	test: 0.6107447	best

In [ ]:
# Catboost_gpu MAE on test set: 0.307

In [76]:
catboost_params_gpu = {'iterations': 200, 'learning_rate': 0.11, 'depth': 10, 'loss_function': 'Logloss', 'random_seed': 42,"task_type":"GPU"}
catboost_model_gpu, catboost_Y_pred_gpu, catboost_mae_gpu, catboost_Y_pred_proba_gpu = Model.catboost(df_train_whole,df_valid,catboost_params_gpu,1,cat_cols)
print("Catboost_gpu MAE on test set:", catboost_mae_gpu)

0:	learn: 0.6697010	test: 0.6725224	best: 0.6725224 (0)	total: 1.41s	remaining: 4m 40s
1:	learn: 0.6509047	test: 0.6560335	best: 0.6560335 (1)	total: 2.85s	remaining: 4m 42s
2:	learn: 0.6364454	test: 0.6432444	best: 0.6432444 (2)	total: 4.25s	remaining: 4m 38s
3:	learn: 0.6253012	test: 0.6338572	best: 0.6338572 (3)	total: 5.64s	remaining: 4m 36s
4:	learn: 0.6166208	test: 0.6269567	best: 0.6269567 (4)	total: 6.9s	remaining: 4m 28s
5:	learn: 0.6087857	test: 0.6204421	best: 0.6204421 (5)	total: 8.12s	remaining: 4m 22s
6:	learn: 0.6024698	test: 0.6154924	best: 0.6154924 (6)	total: 9.3s	remaining: 4m 16s
7:	learn: 0.5959423	test: 0.6098435	best: 0.6098435 (7)	total: 10.6s	remaining: 4m 13s
8:	learn: 0.5911104	test: 0.6055535	best: 0.6055535 (8)	total: 11.8s	remaining: 4m 10s
9:	learn: 0.5866731	test: 0.6017220	best: 0.6017220 (9)	total: 13.3s	remaining: 4m 13s
10:	learn: 0.5826898	test: 0.5982421	best: 0.5982421 (10)	total: 14.7s	remaining: 4m 13s
11:	learn: 0.5797699	test: 0.5959657	best: 

In [117]:
catboost_params_gpu = {'iterations': 200, 'learning_rate': 0.11, 'depth': 10, 'loss_function': 'Logloss', 'random_seed': 42,"task_type":"GPU"}
catboost_model_gpu, catboost_Y_pred_gpu, catboost_mae_gpu, catboost_Y_pred_proba_gpu = Model.catboost(df_train_whole,df_valid,catboost_params_gpu,1,cat_cols)
print("Catboost_gpu MAE on test set:", catboost_mae_gpu)

0:	learn: 0.6702124	test: 0.6724136	best: 0.6724136 (0)	total: 1.79s	remaining: 5m 55s
1:	learn: 0.6521445	test: 0.6566647	best: 0.6566647 (1)	total: 3.2s	remaining: 5m 16s
2:	learn: 0.6375525	test: 0.6446949	best: 0.6446949 (2)	total: 4.58s	remaining: 5m
3:	learn: 0.6261782	test: 0.6341746	best: 0.6341746 (3)	total: 6.09s	remaining: 4m 58s
4:	learn: 0.6160891	test: 0.6249046	best: 0.6249046 (4)	total: 7.52s	remaining: 4m 53s
5:	learn: 0.6076550	test: 0.6179187	best: 0.6179187 (5)	total: 8.93s	remaining: 4m 48s
6:	learn: 0.6005921	test: 0.6121869	best: 0.6121869 (6)	total: 10.2s	remaining: 4m 40s
7:	learn: 0.5956491	test: 0.6085963	best: 0.6085963 (7)	total: 11.6s	remaining: 4m 38s
8:	learn: 0.5910855	test: 0.6051027	best: 0.6051027 (8)	total: 12.8s	remaining: 4m 31s
9:	learn: 0.5870598	test: 0.6021569	best: 0.6021569 (9)	total: 14s	remaining: 4m 26s
10:	learn: 0.5835557	test: 0.5989658	best: 0.5989658 (10)	total: 15.6s	remaining: 4m 27s
11:	learn: 0.5803496	test: 0.5957072	best: 0.595

In [62]:
catboost_params_gpu = {'iterations': 175, 'learning_rate': 0.11, 'depth': 10, 'loss_function': 'Logloss', 'random_seed': 42,"task_type":"GPU"}
catboost_model_gpu, catboost_Y_pred_gpu, catboost_mae_gpu, catboost_Y_pred_proba_gpu = Model.catboost(df_train,df_test,catboost_params_gpu,1,cat_cols)
print("Catboost_gpu MAE on test set:", catboost_mae_gpu)

0:	learn: 0.6754863	test: 0.6776536	best: 0.6776536 (0)	total: 119ms	remaining: 20.7s
1:	learn: 0.6620300	test: 0.6656603	best: 0.6656603 (1)	total: 228ms	remaining: 19.7s
2:	learn: 0.6481558	test: 0.6558743	best: 0.6558743 (2)	total: 332ms	remaining: 19s
3:	learn: 0.6360583	test: 0.6472132	best: 0.6472132 (3)	total: 436ms	remaining: 18.7s
4:	learn: 0.6274132	test: 0.6411325	best: 0.6411325 (4)	total: 532ms	remaining: 18.1s
5:	learn: 0.6205094	test: 0.6365380	best: 0.6365380 (5)	total: 633ms	remaining: 17.8s
6:	learn: 0.6137337	test: 0.6320084	best: 0.6320084 (6)	total: 738ms	remaining: 17.7s
7:	learn: 0.6082135	test: 0.6279272	best: 0.6279272 (7)	total: 837ms	remaining: 17.5s
8:	learn: 0.6026527	test: 0.6239704	best: 0.6239704 (8)	total: 937ms	remaining: 17.3s
9:	learn: 0.5990705	test: 0.6222258	best: 0.6222258 (9)	total: 1.05s	remaining: 17.3s
10:	learn: 0.5947971	test: 0.6183683	best: 0.6183683 (10)	total: 1.14s	remaining: 17s
11:	learn: 0.5914570	test: 0.6159073	best: 0.6159073 (11

In [165]:
# transpose using np
catboost_Y_pred_gpu_array = np.array([catboost_Y_pred_gpu])
catboost_Y_pred_gpu_array_transpose = catboost_Y_pred_gpu_array.transpose()
# to csv
np.savetxt("catboost_Y_pred_gpu_array_transpose.csv", catboost_Y_pred_gpu_array_transpose, delimiter=",")

In [126]:
Model.get_feature_importance(catboost_model_gpu,df_valid)

[('user_id', 12.206029696227548),
 ('delivery_time', 7.592142169203142),
 ('item_id', 6.14959404905104),
 ('delivery_time_item_id_item_color_user_id_std', 3.9434822606038114),
 ('delivery_time_item_id_delivery_weekday_user_id_max', 2.3346432264896606),
 ('delivery_time_delivery_weekday_std', 1.7983701758959179),
 ('days_since_last_order_same_item_id', 1.5636814686643943),
 ('days_since_last_delivery', 1.542499686528102),
 ('order_sum', 1.3241819863934212),
 ('item_price', 1.2399723451231264),
 ('order_number_same_item_id_item_color', 1.1754139510613981),
 ('size', 1.1523748146161785),
 ('days_since_last_delivery_same_item_id', 1.0817581247872403),
 ('order_item_id_size_nunique', 1.0643032164625652),
 ('delivery_time_item_id_size_user_id_mad', 0.9283297604707944),
 ('delivery_time_item_id_delivery_weekday_user_id_min', 0.8966145492078695),
 ('delivery_time_item_id_size_user_id_std', 0.8884928692897951),
 ('delivery_time_size_delivery_weekday_mad', 0.7403540642573762),
 ('delivery_time_o

In [59]:
catboost_params_gpu = {'iterations': 175, 'learning_rate': 0.11, 'depth': 10, 'loss_function': 'Logloss', 'random_seed': 42,"task_type":"GPU"}
catboost_model_gpu, catboost_Y_pred_gpu, catboost_mae_gpu, catboost_Y_pred_proba_gpu = Model.catboost(df_train,df_test,catboost_params_gpu,1,cat_cols)
print("Catboost_gpu MAE on test set:", catboost_mae_gpu)

0:	learn: 0.6762747	test: 0.6782410	best: 0.6782410 (0)	total: 300ms	remaining: 52.3s
1:	learn: 0.6632327	test: 0.6666853	best: 0.6666853 (1)	total: 617ms	remaining: 53.4s
2:	learn: 0.6492923	test: 0.6570831	best: 0.6570831 (2)	total: 910ms	remaining: 52.2s
3:	learn: 0.6370229	test: 0.6483868	best: 0.6483868 (3)	total: 1.21s	remaining: 51.7s
4:	learn: 0.6281548	test: 0.6423957	best: 0.6423957 (4)	total: 1.5s	remaining: 50.9s
5:	learn: 0.6201549	test: 0.6371397	best: 0.6371397 (5)	total: 1.79s	remaining: 50.5s
6:	learn: 0.6139473	test: 0.6325477	best: 0.6325477 (6)	total: 2.1s	remaining: 50.4s
7:	learn: 0.6086409	test: 0.6291967	best: 0.6291967 (7)	total: 2.4s	remaining: 50s
8:	learn: 0.6050084	test: 0.6275087	best: 0.6275087 (8)	total: 2.69s	remaining: 49.6s
9:	learn: 0.6015575	test: 0.6252537	best: 0.6252537 (9)	total: 3.02s	remaining: 49.8s
10:	learn: 0.5989756	test: 0.6238928	best: 0.6238928 (10)	total: 3.29s	remaining: 49.1s
11:	learn: 0.5956252	test: 0.6214219	best: 0.6214219 (11)

In [62]:
catboost_params_gpu = {'iterations': 175, 'learning_rate': 0.11, 'depth': 10, 'loss_function': 'Logloss', 'random_seed': 42,"task_type":"GPU"}
catboost_model_gpu, catboost_Y_pred_gpu, catboost_mae_gpu, catboost_Y_pred_proba_gpu = Model.catboost(df_train_whole,df_valid,catboost_params_gpu,1,cat_cols)
print("Catboost_gpu MAE on test set:", catboost_mae_gpu)

0:	learn: 0.6763689	test: 0.6756236	best: 0.6756236 (0)	total: 706ms	remaining: 2m 2s
1:	learn: 0.6634344	test: 0.6619983	best: 0.6619983 (1)	total: 1.03s	remaining: 1m 29s
2:	learn: 0.6493913	test: 0.6509276	best: 0.6509276 (2)	total: 1.33s	remaining: 1m 16s
3:	learn: 0.6382779	test: 0.6419285	best: 0.6419285 (3)	total: 1.64s	remaining: 1m 9s
4:	learn: 0.6286101	test: 0.6337999	best: 0.6337999 (4)	total: 1.97s	remaining: 1m 6s
5:	learn: 0.6216611	test: 0.6283822	best: 0.6283822 (5)	total: 2.27s	remaining: 1m 3s
6:	learn: 0.6150450	test: 0.6227163	best: 0.6227163 (6)	total: 2.59s	remaining: 1m 2s
7:	learn: 0.6094959	test: 0.6180682	best: 0.6180682 (7)	total: 2.93s	remaining: 1m 1s
8:	learn: 0.6057959	test: 0.6155744	best: 0.6155744 (8)	total: 3.25s	remaining: 60s
9:	learn: 0.6024580	test: 0.6131919	best: 0.6131919 (9)	total: 3.57s	remaining: 58.9s
10:	learn: 0.5998725	test: 0.6115962	best: 0.6115962 (10)	total: 3.87s	remaining: 57.7s
11:	learn: 0.5962936	test: 0.6086550	best: 0.6086550

In [219]:
Model.get_feature_importance(catboost_model_gpu,df_train)

[('delivery_time', 13.952286528488756),
 ('user_id', 10.958014987884432),
 ('item_id', 6.165970067281955),
 ('days_since_last_delivery', 3.7703008919586174),
 ('delivery_time_delivery_weekday_std', 2.6206329938458097),
 ('order_sum', 2.224761203585216),
 ('delivery_time_delivery_weekday_mad', 2.0060063944899214),
 ('order_item_id_size_nunique', 1.5206501343464565),
 ('delivery_time_user_id_min', 1.4380087948019447),
 ('price-item_price_item_id_mean', 1.30610727529431),
 ('delivery_time_user_id_median', 1.1800903928778124),
 ('brand_id', 1.1115892761927098),
 ('item_price', 1.110651868621796),
 ('delivery_weekday', 1.0849852121423664),
 ('item_price_user_id_sum', 1.04342573378681),
 ('order_number_same_item_id_item_color', 1.028889142717541),
 ('days_since_last_order_user_id_min', 1.0155767615775184),
 ('days_since_last_order_same_item_id', 0.9434149308354873),
 ('days_since_last_delivery_user_id_median', 0.8980411893568441),
 ('item_color', 0.8704195882433355),
 ('days_since_last_deliv

In [137]:
xgboost_params = {'max_depth': 9, 'learning_rate': 0.15, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 42,"objective":"binary:logistic"}
xgb_model, xgb_Y_pred, xgb_mae, xgb_Y_pred_proba = Model.xgboost(df_train_encoded,df_test_encoded,xgboost_params,1)
print("Xgboost MAE on test set:", xgb_mae)

[0]	validation_0-mae:0.46576
[1]	validation_0-mae:0.43660
[2]	validation_0-mae:0.41188
[3]	validation_0-mae:0.39129
[4]	validation_0-mae:0.37320
[5]	validation_0-mae:0.35700
[6]	validation_0-mae:0.34347
[7]	validation_0-mae:0.33148
[8]	validation_0-mae:0.32139
[9]	validation_0-mae:0.31233
[10]	validation_0-mae:0.30480
[11]	validation_0-mae:0.29780
[12]	validation_0-mae:0.29223
[13]	validation_0-mae:0.28642
[14]	validation_0-mae:0.28181
[15]	validation_0-mae:0.27795
[16]	validation_0-mae:0.27461
[17]	validation_0-mae:0.27185
[18]	validation_0-mae:0.26835
[19]	validation_0-mae:0.26582
[20]	validation_0-mae:0.26402
[21]	validation_0-mae:0.26229
[22]	validation_0-mae:0.25987
[23]	validation_0-mae:0.25819
[24]	validation_0-mae:0.25671
[25]	validation_0-mae:0.25557
[26]	validation_0-mae:0.25445
[27]	validation_0-mae:0.25305
[28]	validation_0-mae:0.25213
[29]	validation_0-mae:0.25151
[30]	validation_0-mae:0.25055
[31]	validation_0-mae:0.24995
[32]	validation_0-mae:0.24910
[33]	validation_0-ma

In [140]:
xgboost_params = {'max_depth': 9, 'learning_rate': 0.15, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 42,"objective":"binary:logistic"}
xgb_model, xgb_Y_pred, xgb_mae, xgb_Y_pred_proba = Model.xgboost(df_train_encoded,df_test_encoded,xgboost_params,1)
print("Xgboost MAE on test set:", xgb_mae)

[0]	validation_0-mae:0.49593
[1]	validation_0-mae:0.49163
[2]	validation_0-mae:0.49254
[3]	validation_0-mae:0.49093
[4]	validation_0-mae:0.48723
[5]	validation_0-mae:0.48524
[6]	validation_0-mae:0.48412
[7]	validation_0-mae:0.48153
[8]	validation_0-mae:0.47949
[9]	validation_0-mae:0.47737
[10]	validation_0-mae:0.47476
[11]	validation_0-mae:0.47259
[12]	validation_0-mae:0.47016
[13]	validation_0-mae:0.46701
[14]	validation_0-mae:0.46639
[15]	validation_0-mae:0.46435
[16]	validation_0-mae:0.46240
[17]	validation_0-mae:0.46329
[18]	validation_0-mae:0.46249
[19]	validation_0-mae:0.46153
[20]	validation_0-mae:0.46095
[21]	validation_0-mae:0.45929
[22]	validation_0-mae:0.45766
[23]	validation_0-mae:0.45671
[24]	validation_0-mae:0.45588
[25]	validation_0-mae:0.45547
[26]	validation_0-mae:0.45590
[27]	validation_0-mae:0.45557
[28]	validation_0-mae:0.45499
[29]	validation_0-mae:0.45520
[30]	validation_0-mae:0.45489
[31]	validation_0-mae:0.45386
[32]	validation_0-mae:0.45371
[33]	validation_0-ma

In [113]:
xgboost_params = {'max_depth': 9, 'learning_rate': 0.15, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 42,"objective":"binary:logistic"}
xgb_model, xgb_Y_pred, xgb_mae, xgb_Y_pred_proba = Model.xgboost(df_train_encoded,df_test_encoded,xgboost_params,1)
print("Xgboost MAE on test set:", xgb_mae)

Xgboost MAE on test set: 0.3236


In [11]:
mae_dict_1 = {"Catboost_native_CPU":0.3166,"Catboost_native_GPU":0.317,"Xgboost":0.3236}
model_results_1 = pd.DataFrame(mae_dict_1,index=["MAE_4"])
model_results_1.sort_values(by="MAE_4",ascending=True,inplace=True,axis=1)
model_results_1 = model_results_1.transpose()
model_results_1.index.set_names(["Model"],inplace=True)
model_results_1.reset_index(inplace=True)
model_results_1

,Model,MAE_4
0,Catboost_native_CPU,0.3166
1,Catboost_native_GPU,0.3170
2,Xgboost,0.3236


In [12]:
model_performance = pd.read_csv("Model_Performances/Model_performance_1.csv")
model_performance

,Model,MAE_3
0,Catboost_native_GPU,0.3191
1,Catboost_native_CPU,0.3196
2,Xgboost,0.3258
3,Lightgbm_native,0.3301


In [18]:
model_results_whole = pd.merge(model_results_1,model_performance,on="Model",how="left")
model_results_whole["MAE_Improvement"] = model_results_whole["MAE_3"] - model_results_whole["MAE_4"]
model_results_whole = model_results_whole[["Model","MAE_3","MAE_4","MAE_Improvement"]]
model_results_whole

,Model,MAE_3,MAE_4,MAE_Improvement
0,Catboost_native_CPU,0.3196,0.3166,0.0030
1,Catboost_native_GPU,0.3191,0.3170,0.0021
2,Xgboost,0.3258,0.3236,0.0022


In [19]:
# df["order_date"] = pd.to_datetime(df["order_date"], format="%Y-%m-%d")
# df["delivery_date"] = pd.to_datetime(df["delivery_date"], format="%Y-%m-%d")

# # Get the year for order date and delivery date
# df['order_year'] = df['order_date'].dt.year
# df['delivery_year'] = df['delivery_date'].dt.year

# # Get the season for order date and delivery date
# df['order_season'] = df['order_date'].dt.month.apply(lambda x: 'winter' if x in [12,1,2] else 'spring' if x in [3,4,5] else 'summer' if x in [6,7,8] else 'autumn')
# df['delivery_season'] = df['delivery_date'].dt.month.apply(lambda x: 'winter' if x in [12,1,2] else 'spring' if x in [3,4,5] else 'summer' if x in [6,7,8] else 'autumn')

# # Get the day of the year for order date and delivery date
# df['order_day_of_year'] = df['order_date'].dt.dayofyear
# df['delivery_day_of_year'] = df['delivery_date'].dt.dayofyear

In [275]:
interaction_cols = cat_cols.copy()
interaction_cols += ["order_weekday","delivery_weekday","delivery_week","order_item_count","delivery_time"]
# Remove the columns that are not needed for the interaction
interaction_cols = [x for x in interaction_cols if x not in ["mode_item_id","mode_size","mode_brand_id","mode_item_color","user_state","user_title","user_id"]]
interaction_cols

['size',
 'item_color',
 'item_id',
 'brand_id',
 'order_weekday',
 'delivery_weekday',
 'delivery_week',
 'order_item_count',
 'delivery_time']

In [ ]:
# Delivery time, order sum, user_age, item price binning

In [276]:
# !! Include in code base
# Use the numeric variable order_sum to create a new categorical variable using bins
bins_range = df["order_sum"].describe(percentiles=[0.1,0.25,0.5,0.75,0.9])
# bins_range
bins = bins_range[["min","10%","25%","50%","75%","90%","max"]].values
df['order_sum_bins'] = pd.cut(df['order_sum'], bins=bins, include_lowest=True).cat.codes

In [277]:
bins_range = df["item_price"].describe(percentiles=[0.1,0.25,0.5,0.75,0.95])
bins = bins_range[["min","10%","25%","50%","75%","95%","max"]].values
df['item_price_bins'] = pd.cut(df['item_price'], bins=bins, include_lowest=True).cat.codes

In [ ]:
# catboost_cpu_importance_list = [col for tuple in catboost_cpu_importance[:50] for col in tuple]
# # code base convert list of tuples to list
# # out = [item for t in listoftuples for item in t]
# catboost_cpu_importance_list

In [101]:
# df.loc[:, 'order_number_same_item_id':'order_brand_id_item_id_nunique'].columns
# order_number_important_list = []
# for col in df.loc[:, 'order_number_same_item_id':'order_brand_id_item_id_nunique'].columns:
#     if col in catboost_cpu_importance_list:
#         order_number_important_list.append(col)
# order_number_important_list.remove("order_number_different_size_item_color_brand_id")
# order_number_important_list.remove("order_item_id_size_nunique")
# order_number_important_list

['order_number_same_size',
 'order_number_same_item_id_size',
 'order_number_different_item_id_size',
 'order_number_same_item_id_item_color']

In [278]:
order_number_important_list =['order_number_same_size','order_number_same_item_id_size','order_number_different_item_id_size','order_number_same_item_id_item_color']
interaction_cols += order_number_important_list
interaction_cols += ["order_sum_bins","item_price_bins"]


In [279]:
len(interaction_cols)

15

In [280]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(2,3))
interaction_powerset = powerset(interaction_cols)

In [281]:
# Use the powerset to create new columns with the interaction between the categorical variables
powerset_name_list = []
# exclude powerset elements containing both item_id and brand_id
exclude = ["item_id","brand_id"]
for column in interaction_powerset:
    current = list(column)
    print(current)
    column_name = ""
    # sum the strings in the column
    column_name = "_x_".join(current)
    # df[column_name] = df[current].astype(str).sum(axis=1)
    df[column_name] = df[current[0]].astype(str) + "_" + df[current[1]].astype(str)
    df[column_name] = LE.fit_transform(df[column_name])
    # powerset_name_list.append(column_name)

['size', 'item_color']
['size', 'item_id']
['size', 'brand_id']
['size', 'order_weekday']
['size', 'delivery_weekday']
['size', 'delivery_week']
['size', 'order_item_count']
['size', 'delivery_time']
['size', 'order_number_same_size']
['size', 'order_number_same_item_id_size']
['size', 'order_number_different_item_id_size']
['size', 'order_number_same_item_id_item_color']
['size', 'order_sum_bins']
['size', 'item_price_bins']
['item_color', 'item_id']
['item_color', 'brand_id']
['item_color', 'order_weekday']
['item_color', 'delivery_weekday']
['item_color', 'delivery_week']
['item_color', 'order_item_count']
['item_color', 'delivery_time']
['item_color', 'order_number_same_size']
['item_color', 'order_number_same_item_id_size']
['item_color', 'order_number_different_item_id_size']
['item_color', 'order_number_same_item_id_item_color']
['item_color', 'order_sum_bins']
['item_color', 'item_price_bins']
['item_id', 'brand_id']
['item_id', 'order_weekday']
['item_id', 'delivery_weekday']


In [282]:
to_drop =["item_id_x_brand_id","order_sum_bins","item_price_bins"]
df.drop(to_drop, axis=1, inplace=True)
df

,order_item_id,order_date,delivery_date,item_id,size,item_color,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date,return,delivery_time,order_id,user_age,user_reg_age,order_weekday,delivery_weekday,order_month,delivery_month,order_day,delivery_day,order_week,delivery_week,order_item_count,order_sum,average_item_price_order,order_number_same_item_id,order_number_different_item_id,order_number_same_size,order_number_different_size,order_number_same_item_color,order_number_different_item_color,order_number_same_brand_id,order_number_different_brand_id,order_number_same_item_id_size,order_number_different_item_id_size,order_number_same_item_id_item_color,order_number_different_item_id_item_color,order_number_same_size_item_color,order_number_different_size_item_color,order_number_same_size_brand_id,order_number_different_size_brand_id,order_number_same_item_color_brand_id,order_number_different_item_color_brand_id,order_number_same_item_id_size_item_color,order_number_different_item_id_size_item_color,order_number_same_size_item_color_brand_id,order_number_different_size_item_color_brand_id,order_item_id_nunique,order_size_nunique,order_brand_id_nunique,order_item_color_nunique,order_item_id_color_nunique,order_item_id_size_nunique,order_brand_id_color_nunique,order_brand_id_size_nunique,order_brand_id_item_id_nunique,item_price_item_id_mean,item_price_item_id_std,item_price_item_id_min,item_price_item_id_max,item_price_item_id_sum,item_price_item_id_count,item_price_item_id_median,item_price_item_id_mad,item_price_user_id_mean,item_price_user_id_std,item_price_user_id_min,item_price_user_id_max,item_price_user_id_sum,item_price_user_id_count,item_price_user_id_median,item_price_user_id_mad,item_price_brand_id_mean,item_price_brand_id_std,item_price_brand_id_min,item_price_brand_id_max,item_price_brand_id_sum,item_price_brand_id_count,item_price_brand_id_median,item_price_brand_id_mad,price-item_price_item_id_mean,price-item_price_item_id_min,price-item_price_item_id_max,price-item_price_user_id_mean,price-item_price_user_id_min,price-item_price_user_id_max,price-item_price_brand_id_mean,price-item_price_brand_id_min,price-item_price_brand_id_max,mode_item_id,mode_size,mode_brand_id,mode_item_color,delivery_time_item_id_mean,delivery_time_item_id_std,delivery_time_item_id_min,delivery_time_item_id_max,delivery_time_item_id_median,delivery_time_item_id_mad,delivery_time_size_mean,delivery_time_size_std,delivery_time_size_min,delivery_time_size_max,delivery_time_size_median,delivery_time_size_mad,delivery_time_brand_id_mean,delivery_time_brand_id_std,delivery_time_brand_id_min,delivery_time_brand_id_max,delivery_time_brand_id_median,delivery_time_brand_id_mad,delivery_time_item_color_mean,delivery_time_item_color_std,delivery_time_item_color_min,delivery_time_item_color_max,delivery_time_item_color_median,delivery_time_item_color_mad,delivery_time_user_id_mean,delivery_time_user_id_std,delivery_time_user_id_min,delivery_time_user_id_max,delivery_time_user_id_median,delivery_time_user_id_mad,days_since_last_order,days_since_last_order_user_id_mean,days_since_last_order_user_id_median,days_since_last_order_user_id_max,days_since_last_order_user_id_min,days_since_last_order_user_id_std,days_since_last_order_user_id_mad,days_since_last_order_same_item_id,days_since_last_order_same_size,days_since_last_order_same_item_color,days_since_last_order_same_brand_id,days_since_last_order_same_item_id_size,days_since_last_order_same_item_id_item_color,days_since_last_order_same_size_item_color,days_since_last_order_same_size_brand_id,days_since_last_order_same_item_color_brand_id,days_since_last_order_same_item_id_size_item_color,days_since_last_order_same_size_item_color_brand_id,size_x_item_color,size_x_item_id,size_x_brand_id,size_x_order_weekday,size_x_delivery_weekday,size_x_delivery_week,size_x_order_item_count,size_x_delivery_time,size_x_order_number_same_size,size_x_order_number_same_item_id_size,size_x_order_number

In [283]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 531170 entries, 0 to 531169
Columns: 249 entries, order_item_id to order_sum_bins_x_item_price_bins
dtypes: category(1), datetime64[ns](4), float64(72), int32(110), int64(62)
memory usage: 819.0 MB


In [284]:
df_interactions = df.loc[:, "size_x_item_color":].columns


In [285]:
cat_cols += list(df_interactions)

In [58]:
# cat_cols += ['order_number_same_item_id_item_color_x_item_price_bins', 'order_sum_bins_x_item_price_bins']

In [286]:
# Instantiate the model class
Model = Model_class(df)

In [287]:
# Split the data into train and test sets
df_train,df_test, df_valid = Model.split_data(False)

In [263]:
# columns_to_drop = list(df.loc[:,"size_x_item_color":].columns)
# df.drop(columns_to_drop, axis=1, inplace=True)

In [318]:
# Encode the categorical variables using leave-one-out encoding
df_train_encoded,df_test_encoded,encoder = Model.LOE_Encoder(df_train,df_test, cat_cols,0.1,True)

In [322]:
catboost_params_gpu = {'iterations': 150, 'learning_rate': 0.10, 'depth': 10, 'loss_function': 'Logloss', 'random_seed': 42,"task_type":"GPU"}
catboost_model_gpu, catboost_Y_pred_gpu, catboost_mae_gpu, catboost_Y_pred_proba_gpu = Model.catboost(df_train,df_test,catboost_params_gpu,1,cat_cols)
print("Catboost_gpu MAE on test set:", catboost_mae_gpu)

0:	learn: 0.6740070	test: 0.6775695	best: 0.6775695 (0)	total: 384ms	remaining: 57.3s
1:	learn: 0.6601406	test: 0.6644618	best: 0.6644618 (1)	total: 793ms	remaining: 58.7s
2:	learn: 0.6489071	test: 0.6558499	best: 0.6558499 (2)	total: 1.18s	remaining: 57.6s
3:	learn: 0.6351197	test: 0.6460717	best: 0.6460717 (3)	total: 1.53s	remaining: 55.9s
4:	learn: 0.6246662	test: 0.6389197	best: 0.6389197 (4)	total: 1.91s	remaining: 55.4s
5:	learn: 0.6163609	test: 0.6334470	best: 0.6334470 (5)	total: 2.29s	remaining: 54.9s
6:	learn: 0.6097682	test: 0.6289121	best: 0.6289121 (6)	total: 2.74s	remaining: 55.9s
7:	learn: 0.6045880	test: 0.6255690	best: 0.6255690 (7)	total: 3.12s	remaining: 55.4s
8:	learn: 0.5999740	test: 0.6225483	best: 0.6225483 (8)	total: 3.48s	remaining: 54.6s
9:	learn: 0.5961391	test: 0.6207520	best: 0.6207520 (9)	total: 3.95s	remaining: 55.3s
10:	learn: 0.5930044	test: 0.6191206	best: 0.6191206 (10)	total: 4.35s	remaining: 54.9s
11:	learn: 0.5891137	test: 0.6159990	best: 0.6159990

In [320]:
# catboost_model_gpu

In [ ]:
# 0.3266

In [ ]:
# ..
# 149:	learn: 0.5353342	test: 0.5891270	best: 0.5891270 (149)	total: 59s	remaining: 0us
# bestTest = 0.5891269554
# bestIteration = 149
# Catboost_gpu MAE on test set: 0.3169

In [304]:
feature_importance = Model.get_feature_importance(catboost_model_gpu,df_train)

In [315]:
# # Select the features with the importance higher than 0
# important = []
# for i in feature_importance:
#     if i[1] > 0:
#         important.append(i[0])
# important += ["return"]
# cat_cols_important = [col for col in important if col in cat_cols]
# cat_cols_important 

['user_id',
 'delivery_weekday_x_delivery_week',
 'item_id_x_order_number_same_item_id_item_color',
 'delivery_time_x_order_sum_bins',
 'delivery_time_x_item_price_bins',
 'order_weekday_x_delivery_weekday',
 'order_weekday_x_delivery_time',
 'item_color_x_item_id',
 'item_id_x_item_price_bins',
 'delivery_week_x_delivery_time',
 'delivery_weekday_x_delivery_time',
 'size_x_item_id',
 'brand_id_x_order_sum_bins',
 'delivery_weekday_x_order_item_count',
 'delivery_time_x_order_number_same_item_id_size',
 'item_id_x_delivery_week',
 'delivery_week_x_order_item_count',
 'item_id_x_order_number_same_item_id_size',
 'order_number_same_item_id_item_color_x_order_sum_bins',
 'item_id_x_delivery_time',
 'size_x_delivery_time',
 'mode_brand_id',
 'size_x_order_number_same_item_id_size',
 'delivery_weekday_x_order_number_same_item_id_item_color',
 'item_id',
 'brand_id_x_item_price_bins',
 'delivery_time_x_order_number_same_item_id_item_color',
 'order_number_different_item_id_size_x_order_sum_b

In [293]:
catboost_params_gpu = {'iterations': 100, 'learning_rate': 0.10, 'depth': 5, 'loss_function': 'Logloss', 'random_seed': 42,"task_type":"GPU"}
catboost_model_gpu, catboost_Y_pred_gpu, catboost_mae_gpu, catboost_Y_pred_proba_gpu = Model.catboost(df_train[top_100_list],df_test[top_100_list],catboost_params_gpu,1,cat_cols_100)
print("Catboost_gpu MAE on test set:", catboost_mae_gpu)

0:	learn: 0.6798034	test: 0.6813977	best: 0.6813977 (0)	total: 107ms	remaining: 10.6s
1:	learn: 0.6690292	test: 0.6711785	best: 0.6711785 (1)	total: 211ms	remaining: 10.3s
2:	learn: 0.6604074	test: 0.6635671	best: 0.6635671 (2)	total: 325ms	remaining: 10.5s
3:	learn: 0.6530407	test: 0.6568194	best: 0.6568194 (3)	total: 438ms	remaining: 10.5s
4:	learn: 0.6434141	test: 0.6498137	best: 0.6498137 (4)	total: 544ms	remaining: 10.3s
5:	learn: 0.6355052	test: 0.6444124	best: 0.6444124 (5)	total: 660ms	remaining: 10.3s
6:	learn: 0.6289714	test: 0.6402580	best: 0.6402580 (6)	total: 762ms	remaining: 10.1s
7:	learn: 0.6235372	test: 0.6366890	best: 0.6366890 (7)	total: 874ms	remaining: 10.1s
8:	learn: 0.6190288	test: 0.6336324	best: 0.6336324 (8)	total: 987ms	remaining: 9.98s
9:	learn: 0.6150680	test: 0.6311964	best: 0.6311964 (9)	total: 1.09s	remaining: 9.83s
10:	learn: 0.6110241	test: 0.6293643	best: 0.6293643 (10)	total: 1.2s	remaining: 9.72s
11:	learn: 0.6078916	test: 0.6277054	best: 0.6277054 

In [227]:
interactions_importance = Model.get_feature_importance(catboost_model_gpu, df_train[top_100_list])
interactions_importance

[('user_id', 14.679686296203286),
 ('item_id_x_order_number_same_item_id_item_color', 6.3623625554544905),
 ('user_id_x_delivery_time', 5.823748801670317),
 ('delivery_weekday_x_delivery_week', 5.20685469082547),
 ('user_id_x_delivery_weekday', 5.004109147717877),
 ('delivery_time_x_item_price_bins', 4.771916933415711),
 ('item_color_x_item_id', 3.728815861441958),
 ('days_since_last_order_same_item_id', 3.447457598605229),
 ('delivery_time_x_order_sum_bins', 3.2285967363502635),
 ('order_weekday_x_delivery_time', 2.893454610847788),
 ('item_id_x_item_price_bins', 2.5937687137994327),
 ('delivery_weekday_x_order_item_count', 2.36722675770643),
 ('size_x_user_id', 2.0781365380767998),
 ('user_id_x_order_number_same_item_id_size', 1.7780313713646),
 ('delivery_time_x_order_number_same_item_id_size', 1.7635498060953856),
 ('size_x_item_id', 1.5794987972873324),
 ('order_weekday_x_delivery_weekday', 1.4838556894665602),
 ('order_number_same_item_id_item_color_x_order_sum_bins', 1.415062009

In [ ]:
catboost_params_cpu = {'iterations': 150, 'learning_rate': 0.1, 'depth': 10, 'loss_function': 'Logloss', 'random_seed': 42}
catboost_model_cpu, catboost_Y_pred_cpu, catboost_mae_cpu, catboost_Y_pred_proba_cpu = Model.catboost(df_train,df_test,catboost_params_cpu,1,cat_cols)
print("Catboost_cpu MAE on test set:", catboost_mae_cpu)

In [61]:
Model.get_feature_importance(catboost_model_cpu,df_train)

[('order_weekday_x_delivery_weekday', 11.201640533967337),
 ('user_id', 8.230001780102228),
 ('user_id_x_delivery_weekday', 5.720732678615244),
 ('item_id_x_user_id', 4.022871213911953),
 ('delivery_weekday_x_delivery_week', 3.788687216467598),
 ('item_id_x_order_number_same_item_id_item_color', 2.6982403753984636),
 ('delivery_time', 2.258498084497127),
 ('user_id_x_delivery_week', 2.0112232089572997),
 ('item_id_x_item_price_bins', 1.894513848426602),
 ('size_x_user_id', 1.8791192914508645),
 ('user_id_x_order_number_different_item_id_size', 1.839668768234921),
 ('days_since_last_order_same_item_id', 1.814049277174382),
 ('item_color_x_item_id', 1.3987060267641322),
 ('user_id_x_order_item_count', 1.171946088899031),
 ('delivery_weekday_x_item_price_bins', 1.119149239403767),
 ('brand_id_x_order_sum_bins', 1.0970783268453632),
 ('order_item_id_size_nunique', 1.0640502222907942),
 ('size_x_item_id', 1.011583063356824),
 ('item_color_x_user_id', 0.9311620673935717),
 ('item_id_x_order_

In [319]:
xgboost_params = {'max_depth': 9, 'learning_rate': 0.15, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 42,"objective":"binary:logistic"}
xgb_model, xgb_Y_pred, xgb_mae, xgb_Y_pred_proba = Model.xgboost(df_train_encoded,df_test_encoded,xgboost_params,1)
print("Xgboost MAE on test set:", xgb_mae)

[0]	validation_0-mae:0.48896
[1]	validation_0-mae:0.47966
[2]	validation_0-mae:0.47170
[3]	validation_0-mae:0.46497
[4]	validation_0-mae:0.45920
[5]	validation_0-mae:0.45402
[6]	validation_0-mae:0.44893
[7]	validation_0-mae:0.44505
[8]	validation_0-mae:0.44165
[9]	validation_0-mae:0.43867
[10]	validation_0-mae:0.43629
[11]	validation_0-mae:0.43407
[12]	validation_0-mae:0.43224
[13]	validation_0-mae:0.43058
[14]	validation_0-mae:0.42914
[15]	validation_0-mae:0.42778
[16]	validation_0-mae:0.42660
[17]	validation_0-mae:0.42562
[18]	validation_0-mae:0.42467
[19]	validation_0-mae:0.42387
[20]	validation_0-mae:0.42283
[21]	validation_0-mae:0.42199
[22]	validation_0-mae:0.42140
[23]	validation_0-mae:0.42058
[24]	validation_0-mae:0.42014
[25]	validation_0-mae:0.41944
[26]	validation_0-mae:0.41906
[27]	validation_0-mae:0.41879
[28]	validation_0-mae:0.41841
[29]	validation_0-mae:0.41802
[30]	validation_0-mae:0.41771
[31]	validation_0-mae:0.41726
[32]	validation_0-mae:0.41695
[33]	validation_0-ma

In [203]:
Model.get_feature_importance(xgb_model,df_train_encoded)

[('user_id_x_delivery_time_LOE_encoded', 0.07900429),
 ('item_id_x_order_sum_bins_LOE_encoded', 0.046823744),
 ('user_id_x_delivery_weekday_LOE_encoded', 0.046650458),
 ('delivery_time_x_item_price_bins_LOE_encoded', 0.0427944),
 ('order_number_same_item_id_item_color', 0.025408747),
 ('user_id_x_order_number_same_item_id_size_LOE_encoded', 0.019417152),
 ('item_id_x_order_number_same_item_id_item_color_LOE_encoded', 0.017232347),
 ('user_id_LOE_encoded', 0.015677378),
 ('order_number_different_item_id_size_item_color', 0.014801187),
 ('user_id_x_order_number_different_item_id_size_LOE_encoded', 0.013504232),
 ('delivery_weekday_x_delivery_time_LOE_encoded', 0.012400237),
 ('order_item_count', 0.012199569),
 ('order_item_id_size_nunique', 0.010938023),
 ('delivery_weekday_x_delivery_week_LOE_encoded', 0.010710115),
 ('item_id_x_delivery_weekday_LOE_encoded', 0.010326452),
 ('item_id_x_delivery_time_LOE_encoded', 0.010128442),
 ('order_number_same_item_id_size', 0.009940118),
 ('item_id